In [230]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings, sys, os, re
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader

In [62]:
isRawdata = re.compile(r'.*_rawdata.txt')
isPrice = re.compile(r'.*_price.txt')

def dtCnvrt(x):
    return datetime(x.year, x.month, x.day)

priceCols = ['date', 'amount', 'OPEN', 'HIGH', 'LOW', 'CLOSE',]
rawNewCols =['TXID','date','Timestamp','From','To','amount',]
rawCols = ['date','amount']

writePath = './dataset/'
readPath = './dataset/origin/'
tokens=[token+'/' for token in os.listdir(readPath)]

for token in tokens:
    files = os.listdir(readPath+token)
    for file in files:
        if re.match(isPrice, file):
            price = pd.read_csv(readPath+token+file, sep='\t')
            price['date'] = pd.to_datetime(price['DateTime'])
            price['date'] = price['date'].apply(dtCnvrt)
            price = price.rename(columns={'Volume':'amount'}, index=str)
            price.to_hdf(readPath+token+token[:-1]+'_price.h5', token[:-1]+'_price')
        elif re.match(isRawdata, file):
            raw = pd.read_csv(readPath+token+file, sep='\t', header=None)
            raw = raw.rename(columns={origin : col for origin, col in zip(raw.columns, rawNewCols)}, index=str)
            raw.to_hdf(readPath+token+token[:-1]+'_raw.h5', token[:-1]+'_raw')
        else : continue
    rawCnt = raw['amount'].groupby(raw['date']).agg('count')
    rawCnt = rawCnt.reset_index()
    rawCnt['date'] = pd.to_datetime(rawCnt['date'])
    rawCnt['date'] = rawCnt['date'].apply(dtCnvrt)
    rawCnt = rawCnt.rename(columns={'amount':'count'}, index=str)
    locals()[token[:-1]] = pd.merge(price[priceCols], rawCnt, how='left', on='date')
    locals()[token[:-1]].to_hdf(writePath+token[:-1]+'_price.h5', token[:-1]+'_price')

In [68]:
tokens = [token[:-1] for token in tokens]
for token in tokens:
    locals()[token] = locals()[token].dropna()

In [83]:
AE['logAmount'] = np.log(AE['amount'])
features = AE.columns.tolist()[2:]

In [156]:
b.device

device(type='cuda', index=0)

In [293]:
device = torch.device("cuda")
gru = torch.nn.GRU(
    input_size=6,
    hidden_size=1,
    num_layers =6,
    bias=True,
    batch_first = False,
    dropout = 0.2,
    bidirectional=False,
)
a = AE[features].values
a = np.reshape(a, (-1,1,6))
a = torch.tensor(a, dtype=torch.float32, device=device)
a = Variable(a, requires_grad=True)
b = torch.randn(6, 1, 1, dtype=torch.float32, device=device)
b = Variable(b, requires_grad=True)
if torch.cuda.is_available():
    a = a.cuda()
    b = b.cuda()
    gru = gru.cuda()
    output, hn = gru(a, b)

In [267]:
d = torch.randn(6, 6, 6, dtype=torch.float32, device=device)


tensor([[ 0.4974, -2.1601,  0.2941, -0.8790, -2.4491,  0.6189],
        [-1.3471,  1.4144,  1.3748,  0.5732, -1.2410, -1.1691],
        [ 0.9159, -0.4909, -1.0139,  0.3838, -0.9801,  0.7376],
        [-0.2673, -0.2613,  2.4366, -0.4863, -1.4837, -0.9447],
        [ 0.5382, -0.4797,  0.8904,  2.3218,  0.7024,  0.6615],
        [-1.0265, -0.0446,  0.8256,  0.5002,  2.0203,  1.8613]],
       device='cuda:0')

## create NN

In [304]:
class my_model(nn.Module):
    def __init__(self):
        super(my_model, self).__init__()
        self.gru_1 = nn.GRU(input_size=6,hidden_size=6,num_layers =6,dropout = 0.5).cuda()
        self.gru_2 = nn.GRU(input_size=6,hidden_size=6,num_layers =6,dropout = 0.5).cuda()
        self.gru_3 = nn.GRU(input_size=6,hidden_size=1,num_layers =6,dropout = 0.5).cuda()
    def forward(self, input_, h0,h1,h2):
        ret, hn0 = self.gru_1(input_.cuda(), h0.cuda())
        ret, hn1 = self.gru_2(ret.cuda(), h1.cuda())
        ret, hn2 = self.gru_3(ret.cuda(), h2.cuda())
        return ret, hn0, hn1, hn2

In [297]:
if torch.cuda.is_available():
    a = torch.tensor(c.next().reshape(1,6,6),dtype=torch.float32, device=device)
    a = Variable(a, requires_grad=True).cuda()
    b = Variable(torch.randn(6, 6, 1,dtype=torch.float32, device=device), requires_grad=True).cuda()

In [301]:
a = torch.tensor(c.next().reshape(1,6,6),dtype=torch.float32, device=device)
a = Variable(a, requires_grad=True)
b0 = Variable(torch.randn(6, 6, 6,dtype=torch.float32, device=device), requires_grad=True)
b1 = Variable(torch.randn(6, 6, 6,dtype=torch.float32, device=device), requires_grad=True)
b2 = Variable(torch.randn(6, 6, 1,dtype=torch.float32, device=device), requires_grad=True)
m = my_model()
out = m(a, b0, b1, b2)

In [302]:
out

tensor([[[ 0.6855],
         [-0.9100],
         [ 0.2637],
         [-0.3124],
         [-0.5991],
         [-0.8155]]], device='cuda:0', grad_fn=<CudnnRnnBackward>)

## train

In [355]:
m = my_model()
adam = torch.optim.Adam(m.parameters(),lr=0.1)
mse = nn.MSELoss()
b0 = Variable(torch.randn(6, 6, 6,dtype=torch.float32, device=device), requires_grad=True)
b1 = Variable(torch.randn(6, 6, 6,dtype=torch.float32, device=device), requires_grad=True)
b2 = Variable(torch.randn(6, 6, 1,dtype=torch.float32, device=device), requires_grad=True)

In [356]:
label = [1.,1.,1.,1.,1.,1.,]
label = Variable(torch.tensor(label,dtype=torch.float32, device=device), requires_grad=True).cuda()

In [350]:
data = c.next()

In [358]:
out

tensor([[[0.9998],
         [0.9999],
         [0.9999],
         [0.9999],
         [0.9998],
         [0.9831]]], device='cuda:0', grad_fn=<CudnnRnnBackward>)

In [357]:
for _ in range(1000):
    data = torch.tensor(data.reshape(1,6,6),dtype=torch.float32, device=device)
    data = Variable(data, requires_grad=True).cuda()
    adam.zero_grad()
    out, b0, b1, b2 = m(data, b0, b1, b2,)
    out, b0, b1, b2 = out.cuda(), b0.cuda(), b1.cuda(), b2.cuda()
    loss = mse(out, label)
    loss.backward(retain_graph=True)
    adam.step()
    print(loss)

tensor(1.9169, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(1.7328, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(1.3700, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(1.0227, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.5727, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.2827, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.2139, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.1057, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.0287, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.0386, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.0123, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.0153, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.0051, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.0003, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.0044, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.0019, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.0070, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.0002, device='cuda:0',

tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(4.9499e-05, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(4.8977e-05, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.0002, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.0002, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(1.8692e-06, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(4.7105e-08, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(4.6082e-05, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.0002, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.0002, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(1.8162e-06, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(5.2195e-05, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(5.3877e-07, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(4.8427e-05, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(8.7448e-05, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(9.0810e-07, device='cuda:0', grad_fn=<Mea

KeyboardInterrupt: 